In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.T

In [ ]:
df.describe

In [ ]:
df.dtypes

In [ ]:
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors="coerce")

In [ ]:
df.isnull().sum()

In [ ]:
df.TotalCharges = df.TotalCharges.fillna(0)  #eksik verilere 0 değerini atıyoruz 11 adet eksik değer olduğunu yukardakı satırdan görebiliriz.

In [ ]:
df.columns = df.columns.str.lower().str.replace(" ", "_") #sutun isimlerini kucuk harfe çevirdik boşluk varsa _ koyduk

In [ ]:
df.head(3)

In [ ]:
string_columns  = list(df.dtypes[df.dtypes=="object"].index)
string_columns

In [ ]:
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(" ","_")

In [ ]:
df.head(3)

In [ ]:
df.churn.head()

In [ ]:
df.churn = (df.churn == "yes").astype(int )  #churn sutununda yes ifadesine eşit olanlar 1 geri kalan 0 olacak şekilde veri manipilasyonu

In [ ]:
df.churn.head()

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train_full , df_test = train_test_split(df,test_size=0.2,random_state= 42)

In [ ]:
df_train , df_val = train_test_split(df_train_full , test_size =0.25,random_state= 42)

In [ ]:
y_train =df_train.churn.values

In [ ]:
y_val = df_val.churn.values

In [ ]:
del df_train["churn"]
del df_val["churn"]

In [ ]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',  
               'phoneservice', 'multiplelines', 'internetservice',  
               'onlinesecurity', 'onlinebackup', 'deviceprotection',  
               'techsupport', 'streamingtv', 'streamingmovies',  'contract', 
               'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']  #one-hot-encoder işlemi yapmak için bunları belirlememiz lazım

In [ ]:
train_dict = df_train[categorical + numerical].to_dict(orient = "records")
train_dict[:1]

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse = False)

In [ ]:
dv.fit(train_dict)

In [ ]:
X_train = dv.transform(train_dict)

In [ ]:
X_train[0]

In [ ]:
dv.get_feature_names_out()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver = "liblinear",random_state=42)
model.fit(X_train,y_train)


In [ ]:
val_dict = df_val[categorical+numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)
y_pred[:5]

In [ ]:
print("The performance of the model on the validation dataset: ",model.score(X_val, y_val))
print("The performance of the model on the training dataset: ",model.score(X_train, y_train))

In [ ]:
print("Bias: ",model.intercept_[0])   #sabit sayıyı bulduk.
print(dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3))))  #bazı katsayıları da bu şekilde görebiliriz.

In [ ]:
musteri = {
 'customerid': '8879-zkjof',
 'gender': 'female',
 'seniorcitizen': 1,
 'partner': 'yes',
 'dependents': 'yes',
 'tenure': 41,
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'one_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'monthlycharges': 99.85,
 'totalcharges': 2990.75,
}
x_new = dv.transform([musteri])
model.predict_proba(x_new)